In [ ]:
!pip install -U openai-whisper
!pip install srt
!pip install googletrans==4.0.0rc1

In [ ]:
import subprocess
import whisper
from datetime import timedelta
import srt
from srt import Subtitle


# 音声データ抽出 (音声ファイル名変更)
subprocess.run(['ffmpeg', '-i', 'test.mp4', '-acodec', 'libmp3lame', '-ab', '256k', 'audio.mp3'])

# Whisperモデル読み込み
model = whisper.load_model("medium")

# 音声ファイル読み込み
audio = whisper.load_audio("audio.mp3")

# 音声認識 (開始・終了時間付きセグメント取得)
result = model.transcribe(audio, verbose=True)
segments = result["segments"]

# SRTファイル用の空リスト初期化
subtitles = []

# セグメントごとにSRT行を作成
for index, segment in enumerate(segments, start=1):
    start_time = timedelta(seconds=segment["start"])
    end_time = timedelta(seconds=segment["end"])
    text = segment["text"]

    subtitle = Subtitle(
        index=index,
        start=start_time,
        end=end_time,
        content=text
    )
    subtitles.append(subtitle)

# SRTファイル内容の作成
srt_content = srt.compose(subtitles)

# ファイルへの書き込み (エンコーディング指定)
with open("output.srt", "w", encoding="utf-8") as f:
    f.write(srt_content)

# SRTファイル読み込み (エンコーディング指定)
with open('output.srt', 'r', encoding='utf-8') as file:
    srt_data = file.read()

In [ ]:
import re
import time
from googletrans import Translator
from requests.exceptions import ReadTimeout

# 翻訳者を初期化します
translator = Translator()

def safe_translate(text, src='en', dest='ja', max_attempts=5):
    for attempt in range(max_attempts):
        try:
            # 翻訳を実行し、成功したら結果を返す
            return translator.translate(text, src=src, dest=dest)
        except ReadTimeout:
            # タイムアウトエラーが発生した場合、指定された回数だけリトライする
            if attempt < max_attempts - 1:
                time.sleep((attempt + 1) * 0.5)  # エラー後の待機時間を徐々に増やす
            else:
                raise  # 最後の試行でもエラーが発生した場合は例外を投げる

# ファイルを開きます
with open('output.srt', 'r', encoding='utf-8') as file:
    lines = file.readlines()

# 翻訳ファイルを開きます
with open('translated_file.srt', 'w', encoding='utf-8') as outfile:
    buffer_lines = []  # テキスト行を貯めるバッファ
    for line in lines:
        if re.match(r'^\d+$', line) or '-->' in line:
            outfile.write(line)
        else:
            line = line.strip()
            if line:
                buffer_lines.append(line)
                if len(buffer_lines) >= 40:  # バッファサイズを40行に設定
                    text_to_translate = "\n".join(buffer_lines)
                    translation = safe_translate(text_to_translate)
                    translated_text = translation.text.split('\n')
                    print(f'Original: {text_to_translate}\nTranslated: {translation.text}\n')
                    for translated_line in translated_text:
                        outfile.write(translated_line + '\n')
                    buffer_lines = []
            else:
                if buffer_lines:
                    text_to_translate = "\n".join(buffer_lines)
                    translation = safe_translate(text_to_translate)
                    translated_text = translation.text.split('\n')
                    print(f'Original: {text_to_translate}\nTranslated: {translation.text}\n')
                    for translated_line in translated_text:
                        outfile.write(translated_line + '\n')
                    buffer_lines = []
                outfile.write('\n')
        time.sleep(0.1)  # リクエスト間の待機時間を0.1秒に設定

In [ ]:
subprocess.run(['ffmpeg', '-i', 'test.mp4', '-vf', "subtitles=translated_file.srt:force_style='FontName=Helvetica,FontSize=12'", 'test2_subtitle.mp4'])